# Stærðfræði og reiknifræði – Skiladæmi 7

In [2]:
#BYRJA -- Keyrið til að frumstilla.
import numpy as np, numpy.random as npr, numpy.linalg as la
import scipy.stats as stat, statsmodels.api as sm
import matplotlib.pyplot as plt
plt.rc('axes', axisbelow=True)
%matplotlib inline
# disp(x,y...) skrifar x,y... með 3 aukastöfum
def disp(*args): print(*(f'{a:.3f}' if isinstance(a,float) else a for a in args))
np.set_printoptions(precision=4, floatmode='fixed', suppress=True)

In [35]:
x=npr.rand(10)
y=npr.rand(10)
disp(x)
(r,p)=stat.pearsonr(x,y)
disp(r)
A = np.c_[x,y]
A
R = np.corrcoef(np.c_[x,y])
A
np.corrcoef(x,y)

[0.3030 0.5264 0.6619 0.0599 0.8301 0.8270 0.3681 0.1242 0.1930 0.7548]
0.371


array([[1.0000, 0.3714],
       [0.3714, 1.0000]])

## 1. Ýmsar viguraðgerðir
Lát $n = 100$ og $x = (1,\ldots,n)$ og $y = (1, 1/2,\ldots,1/n)$. Reiknið í Python:

**a)** Lengd $x$, meðaltal, staðalfrávik og dreifni<br>
**b)** Fjarlægð, fylgni og horn milli $x$ og $y$<br>
**c)** Fjarlægðina milli Reykjavíkur og Akureyrar í beinni loftlínu

*Ef þið finnið hnattstöðuna með Google maps er hún gefin sem tugabrot en ef þið flettið henni upp á netinu er ekki ólíklegt að þið fáið gráður, mínútur og sekúndur, t.d. 64° 15´ 30´´ sem (ef við sleppum sekúndunum) jafngildir 64 + 15/60 = 64.25°*

## 2. Lágmörkun með stigli
**a)** Í skiladæmi 5.B1 var stigull (*gradient*) Rosenbrock-fallsins, $f(x, y) = (1 - x)^2 + 100(y - x^2)^2$ reiknaður og útkoman var 

$$\nabla f(x,y) = \pmatrix{2x - 2 - 400x(y - x^2) \\ 200(y - x^2)}$$

Ef við umritum formúlurnar með því að skrifa $(x_0, y_0)$ í stað $(x,y)$ og skilgreina $x = (x_0, y_0)$ fæst

$$\begin{align}f(x) &= (1-x_0)^2 + 100(x_1 - x_0^2)^2 \\
        \nabla f(x) &= \pmatrix{2x_0 - 2 - 400x_0(x_1 - x_0^2) \\ 200(x_1 - x_0^2)}\end{align}$$
        
Skrifið Python föll `f(x)` og `g(x)` sem reikna fallsgildi og stigul Rosenbrock-fallsins.

**b)** `opt.minimize` hefur valkvæðan stika `jac` sem er nafn falls sem reiknar stigul fallsins sem á að lágmarka (*jac* er stytting á *Jacobian*, sem fyrir $f\colon\Bbb{R}\to\Bbb{R^2}$ er samheiti við *gradient*). Með *jac*-stika verður lágmörkunarkallið:

        result = opt.minimize(f, x0, jac=g)
        
Ef `opt.minimize` hefur aðgang að stiglinum þá þarf mun færri köll á fallið til að finna lággildið. Ákvarðið lággildi Rosenbrock-fallsins ef byrjað er í $x_0=(-1.2, 1)$ bæði með og án stigul-falls, og finnið út hve margar ítrekanir og köll á `f` þarf í hvoru tilviki fyrir sig.

## 3. Næsti nágranni
Þetta dæmi tengist greinunum *Nearest neighbor* og *Document dissimilarity* á bls. 50–51 í kennslubók. Í skránni `wikitíðni.npz` (*npz* er sérstakt Numpy-zip-snið) er búið að safna saman upplýsingum um tíðni 1000 orða í 300 Wikipedia-greinum sem skiptast í 5 flokka (Pokemon, veðurfræði, listir, stofnanir Sameinuðu þjóðanna, og rafmagnsverkfræði). Algengum orðum s.s. "the", "in", "of" hefur verið sleppt. Í heimadæmum 8 verður reynt að finna í hvaða flokki hver grein er, en hér látum við duga að kynnast gögnunum örlítið og æfa okkur í að finna næsta granna. Gögnin má lesa inn og skoða gróflega með:

In [6]:
np.set_printoptions(precision=3)
data = np.load('wikitíðni.npz')
orð = data["dictionary"]
greinar = data["article_titles"]
tíðni = data["article_histograms"]

disp('Fjöldi orða:  ', len(orð))
disp('Fjöldi greina:', len(greinar))
disp('Stærð fylkis: ', np.shape(tíðni))
disp(greinar[::20])    # tuttugasta hver grein
disp(orð[-20:])        # síðustu tuttugu orðin
disp(tíðni[:9,:8])     # tíðni fyrstu 9 orðanna í fyrstu 10 greinunum
x0 = tíðni[0]          # 1000-staka vigur með orðtíðni í fyrstu greininni
x1 = tíðni[1]          # 1000-staka vigur með orðtíðni í annarri greininni
s = sum(tíðni,0)       # 1000-staka vigur með summu staka í hverjum dálki fylkisins
I = np.argsort(x0)     # númer orða í fyrstu greininni í röð ef eftir algengi
disp('||x0 - x1|| =', la.norm(x0-x1))   # fjarlægð milli x0 og x1
disp('Algengustu orð í grein 0:', orð[I[-10:]])
sum(x0)

Fjöldi orða:   1000
Fjöldi greina: 300
Stærð fylkis:  (300, 1000)
['A Bar at the Folies-Bergère' 'Baroclinity' 'Complementary colors'
 'Effect of sun angle on climate' 'Frequency-shift keying'
 'Ice accretion indicator' 'International Trade Centre'
 'Luncheon of the Boating Party' 'Neo-impressionism' 'Plant'
 'Precipitation' 'Space weather' 'Transmitter'
 'United Nations Industrial Development Organization' 'Weather map']
['wet' 'white' 'wide' 'widely' 'wind' 'winds' 'winter' 'wire' 'wireless'
 'women' 'working' 'works' 'worldwide' 'wrote' 'wto' 'year' 'yellow'
 'york' 'young' 'zero']
[[0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.017 0.000 0.000 0.000 0.000 0.000 0.012]
 [0.000 0.012 0.000 0.005 0.000 0.015 0.000 0.006]
 [0.000 0.000 0.032 0.000 0.000 0.000 0.000 0.039]
 [0.000 0.019 0.034 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.025 0.000 0.000 0.000 0.000 0.012 0.000]


4.963669121768596

**a)** Skrifið út síðustu 20 greinarnar og 20. hvert orð<br>
**b)** Um hvað er grein númer 0?<br>
**c)** Finnið titil greinarinnar sem er næst grein 0 (*nearest neighbour*) ef við mælum fjarlægð með normi af mismun milli orðtíðnivigra (gildir 2 stig).<br>
**d)** Finnið þau 10 orð sem hafa hæstu samanlagða tíðnina (notið `np.argsort` og vigurinn `s`)<br>
**e)** Notið `plt.spy` með `figsize=(1,5)` til að skoða fylkið `tíðni`.

In [4]:
help(plt.spy)

Help on function spy in module matplotlib.pyplot:

spy(Z, precision=0, marker=None, markersize=None, aspect='equal', origin='upper', **kwargs)
    Plot the sparsity pattern of a 2D array.
    
    This visualizes the non-zero values of the array.
    
    Two plotting styles are available: image and marker. Both
    are available for full arrays, but only the marker style
    works for `scipy.sparse.spmatrix` instances.
    
    **Image style**
    
    If *marker* and *markersize* are *None*, `~.Axes.imshow` is used. Any
    extra remaining kwargs are passed to this method.
    
    **Marker style**
    
    If *Z* is a `scipy.sparse.spmatrix` or *marker* or *markersize* are
    *None*, a `~matplotlib.lines.Line2D` object will be returned with
    the value of marker determining the marker type, and any
    remaining kwargs passed to `~.Axes.plot`.
    
    Parameters
    ----------
    Z : array-like (M, N)
        The array to be plotted.
    
    precision : float or 'present', opt